In [1]:
import torch
from fastai.vision.all import *
import gc 
import numpy as np
import pandas as pd
import os
from PIL import Image
import PIL
PIL.Image.MAX_IMAGE_PIXELS = None

import torch
import torch.nn.functional as F
from torch import nn

In [2]:
path_labels = np.load('C:/Users/USRx/Desktop/Daria/attr_img_shorted_delattr.npy',allow_pickle=True)
df = pd.DataFrame(path_labels, columns=['path','label'])

In [3]:
df['path'] = df['path'].apply(lambda x: os.path.normpath(x))
df['path'] = df['path']+'.jpg'

In [4]:
df.shape

(289222, 2)

In [5]:
""" dblock = DataBlock(
    blocks=(ImageBlock,MultiCategoryBlock),
    splitter=RandomSplitter(valid_pct=0.3,seed=42),
    get_x=ColReader('path'),
    get_y=ColReader('label',label_delim=','),
    item_tfms = Resize(224),
    batch_tfms=aug_transforms(size=224)
)
dls = dblock.dataloaders(df, bs=64,shuffle=True) """

" dblock = DataBlock(\n    blocks=(ImageBlock,MultiCategoryBlock),\n    splitter=RandomSplitter(valid_pct=0.3,seed=42),\n    get_x=ColReader('path'),\n    get_y=ColReader('label',label_delim=','),\n    item_tfms = Resize(224),\n    batch_tfms=aug_transforms(size=224)\n)\ndls = dblock.dataloaders(df, bs=64,shuffle=True) "

In [6]:
""" wd = 5e-7
opt_func = partial(ranger, wd=wd)
metrics=[FBetaMulti(2.0, 0.2, average='samples'), partial(accuracy_multi, thresh=0.2)] """

" wd = 5e-7\nopt_func = partial(ranger, wd=wd)\nmetrics=[FBetaMulti(2.0, 0.2, average='samples'), partial(accuracy_multi, thresh=0.2)] "

In [7]:
""" learn = vision_learner(dls, resnet34, loss_func=BCEWithLogitsLossFlat(), metrics=metrics, opt_func=opt_func).to_fp16()
learn.fine_tune(10)
learn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=0.2), metrics=metrics, opt_func=opt_func).to_fp16()
learn.fine_tune(10)
learn = vision_learner(dls, mobilenet_v3_large, loss_func=BCEWithLogitsLossFlat(thresh=0.2), metrics=metrics, opt_func=opt_func).to_fp16()
learn.fine_tune(10) """

' learn = vision_learner(dls, resnet34, loss_func=BCEWithLogitsLossFlat(), metrics=metrics, opt_func=opt_func).to_fp16()\nlearn.fine_tune(10)\nlearn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=0.2), metrics=metrics, opt_func=opt_func).to_fp16()\nlearn.fine_tune(10)\nlearn = vision_learner(dls, mobilenet_v3_large, loss_func=BCEWithLogitsLossFlat(thresh=0.2), metrics=metrics, opt_func=opt_func).to_fp16()\nlearn.fine_tune(10) '

BCEWithLogitsLossFlat + все оптимизаторы

In [8]:
""" learn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=0.2), metrics=metrics, opt_func=partial(Adam, wd=wd)).to_fp16()
learn.fine_tune(3)"""

' learn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=0.2), metrics=metrics, opt_func=partial(Adam, wd=wd)).to_fp16()\nlearn.fine_tune(3)'

In [9]:
""" learn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=0.2), metrics=metrics, opt_func=partial(RMSProp, wd=wd)).to_fp16()
learn.fine_tune(3) """

' learn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=0.2), metrics=metrics, opt_func=partial(RMSProp, wd=wd)).to_fp16()\nlearn.fine_tune(3) '

In [10]:
"""learn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=0.2), metrics=metrics, opt_func=partial(ranger, wd=wd)).to_fp16()
learn.fine_tune(3) """

'learn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=0.2), metrics=metrics, opt_func=partial(ranger, wd=wd)).to_fp16()\nlearn.fine_tune(3) '

CustomCrossEntropyLoss + все оптимизаторы

In [11]:
class CustomCrossEntropyLoss(nn.Module):
    def __init__(self):
        super(CustomCrossEntropyLoss, self).__init__()

    def forward(self, preds, targets):
        # Targets are in one-hot encoding
        log_preds = F.log_softmax(preds, dim=1)
        loss = -torch.sum(log_preds * targets, dim=1).mean()
        return loss

In [12]:
""" learn = vision_learner(dls, resnet50, loss_func=CustomCrossEntropyLoss(), metrics=metrics, opt_func=partial(Adam, wd=wd)).to_fp16()
learn.fine_tune(3) """

' learn = vision_learner(dls, resnet50, loss_func=CustomCrossEntropyLoss(), metrics=metrics, opt_func=partial(Adam, wd=wd)).to_fp16()\nlearn.fine_tune(3) '

In [13]:
""" learn = vision_learner(dls, resnet50, loss_func=CustomCrossEntropyLoss(), metrics=metrics, opt_func=partial(RMSProp, wd=wd)).to_fp16()
learn.fine_tune(3) """

' learn = vision_learner(dls, resnet50, loss_func=CustomCrossEntropyLoss(), metrics=metrics, opt_func=partial(RMSProp, wd=wd)).to_fp16()\nlearn.fine_tune(3) '

In [14]:
""" learn = vision_learner(dls, resnet50, loss_func=CustomCrossEntropyLoss(), metrics=metrics, opt_func=partial(ranger, wd=wd)).to_fp16()
learn.fine_tune(3) """

' learn = vision_learner(dls, resnet50, loss_func=CustomCrossEntropyLoss(), metrics=metrics, opt_func=partial(ranger, wd=wd)).to_fp16()\nlearn.fine_tune(3) '

FocalLoss + оптимизаторы

In [15]:
""" learn = vision_learner(dls, resnet50, loss_func=FocalLoss(), metrics=metrics, opt_func=partial(Adam, wd=wd)).to_fp16()
learn.fine_tune(3) """

' learn = vision_learner(dls, resnet50, loss_func=FocalLoss(), metrics=metrics, opt_func=partial(Adam, wd=wd)).to_fp16()\nlearn.fine_tune(3) '

In [16]:
""" learn_2 = vision_learner(dls, resnet50, loss_func=FocalLoss(), metrics=metrics, opt_func=partial(RMSProp, wd=wd)).to_fp16()
learn_2.fine_tune(3) """

' learn_2 = vision_learner(dls, resnet50, loss_func=FocalLoss(), metrics=metrics, opt_func=partial(RMSProp, wd=wd)).to_fp16()\nlearn_2.fine_tune(3) '

In [17]:
""" learn = vision_learner(dls, resnet50, loss_func=FocalLoss(), metrics=metrics, opt_func=partial(ranger, wd=wd)).to_fp16()
learn.fine_tune(3) """

' learn = vision_learner(dls, resnet50, loss_func=FocalLoss(), metrics=metrics, opt_func=partial(ranger, wd=wd)).to_fp16()\nlearn.fine_tune(3) '

LabelSmoothingBCEWithLogitsLossFlat + оптимизаторы

In [18]:
class LabelSmoothingBCEWithLogitsLossFlat(BCEWithLogitsLossFlat):
    def __init__(self, smoothing: float = 0.1, thresh: float = 0.2, **kwargs):
        super().__init__(thresh=thresh, **kwargs)
        self.smoothing = smoothing
        self.thresh = thresh
    
    def __call__(self, inp: torch.Tensor, targ: torch.Tensor, **kwargs) -> torch.Tensor:
        targ_smooth = targ.float() * (1. - self.smoothing) + 0.5 * self.smoothing
        loss = super().__call__(inp, targ_smooth, **kwargs)
        return loss
    
    def predict(self, inp: torch.Tensor) -> torch.Tensor:
        return (torch.sigmoid(inp) > self.thresh).float()
    
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}(smoothing={self.smoothing}, thresh={self.thresh})"

In [19]:
""" learn = vision_learner(dls, resnet50, loss_func=LabelSmoothingBCEWithLogitsLossFlat(), metrics=metrics, opt_func=partial(Adam, wd=wd)).to_fp16()
learn.fine_tune(3) """

' learn = vision_learner(dls, resnet50, loss_func=LabelSmoothingBCEWithLogitsLossFlat(), metrics=metrics, opt_func=partial(Adam, wd=wd)).to_fp16()\nlearn.fine_tune(3) '

In [20]:
""" learn = vision_learner(dls, resnet50, loss_func=LabelSmoothingBCEWithLogitsLossFlat(), metrics=metrics, opt_func=partial(RMSProp, wd=wd)).to_fp16()
learn.fine_tune(3) """

' learn = vision_learner(dls, resnet50, loss_func=LabelSmoothingBCEWithLogitsLossFlat(), metrics=metrics, opt_func=partial(RMSProp, wd=wd)).to_fp16()\nlearn.fine_tune(3) '

In [21]:
""" learn = vision_learner(dls, resnet50, loss_func=LabelSmoothingBCEWithLogitsLossFlat(), metrics=metrics, opt_func=partial(ranger, wd=wd)).to_fp16()
learn.fine_tune(3) """

' learn = vision_learner(dls, resnet50, loss_func=LabelSmoothingBCEWithLogitsLossFlat(), metrics=metrics, opt_func=partial(ranger, wd=wd)).to_fp16()\nlearn.fine_tune(3) '

Изменение параметров wd и thresh

In [22]:
""" WD = [5e-8,5e-7,5e-6,5e-5,5e-9]
THRESH = [0.1,0.15,0.2,0.25,0.05,0.075,0.125]
BETA = 2 """


' WD = [5e-8,5e-7,5e-6,5e-5,5e-9]\nTHRESH = [0.1,0.15,0.2,0.25,0.05,0.075,0.125]\nBETA = 2 '

In [23]:
""" CHOICE = 0
metrics=[FBetaMulti(BETA, THRESH[CHOICE], average='samples'), partial(accuracy_multi, thresh=THRESH[CHOICE])]
learn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=THRESH[CHOICE]), metrics=metrics, opt_func=partial(RMSProp, wd=WD[0])).to_fp16()
learn.fine_tune(3) """

" CHOICE = 0\nmetrics=[FBetaMulti(BETA, THRESH[CHOICE], average='samples'), partial(accuracy_multi, thresh=THRESH[CHOICE])]\nlearn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=THRESH[CHOICE]), metrics=metrics, opt_func=partial(RMSProp, wd=WD[0])).to_fp16()\nlearn.fine_tune(3) "

In [24]:
""" CHOICE = 1
metrics=[FBetaMulti(BETA, THRESH[CHOICE], average='samples'), partial(accuracy_multi, thresh=THRESH[CHOICE])]
learn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=THRESH[CHOICE]), metrics=metrics, opt_func=partial(RMSProp, wd=WD[0])).to_fp16()
learn.fine_tune(3) """

" CHOICE = 1\nmetrics=[FBetaMulti(BETA, THRESH[CHOICE], average='samples'), partial(accuracy_multi, thresh=THRESH[CHOICE])]\nlearn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=THRESH[CHOICE]), metrics=metrics, opt_func=partial(RMSProp, wd=WD[0])).to_fp16()\nlearn.fine_tune(3) "

In [25]:
""" CHOICE = 2
metrics=[FBetaMulti(BETA, THRESH[CHOICE], average='samples'), partial(accuracy_multi, thresh=THRESH[CHOICE])]
learn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=THRESH[CHOICE]), metrics=metrics, opt_func=partial(RMSProp, wd=WD[0])).to_fp16()
learn.fine_tune(3) """

" CHOICE = 2\nmetrics=[FBetaMulti(BETA, THRESH[CHOICE], average='samples'), partial(accuracy_multi, thresh=THRESH[CHOICE])]\nlearn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=THRESH[CHOICE]), metrics=metrics, opt_func=partial(RMSProp, wd=WD[0])).to_fp16()\nlearn.fine_tune(3) "

In [26]:
""" CHOICE = 3
metrics=[FBetaMulti(BETA, THRESH[CHOICE], average='samples'), partial(accuracy_multi, thresh=THRESH[CHOICE])]
learn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=THRESH[CHOICE]), metrics=metrics, opt_func=partial(RMSProp, wd=WD[0])).to_fp16()
learn.fine_tune(3) """

" CHOICE = 3\nmetrics=[FBetaMulti(BETA, THRESH[CHOICE], average='samples'), partial(accuracy_multi, thresh=THRESH[CHOICE])]\nlearn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=THRESH[CHOICE]), metrics=metrics, opt_func=partial(RMSProp, wd=WD[0])).to_fp16()\nlearn.fine_tune(3) "

In [27]:
""" CHOICE = 4
metrics=[FBetaMulti(BETA, THRESH[CHOICE], average='samples'), partial(accuracy_multi, thresh=THRESH[CHOICE])]
learn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=THRESH[CHOICE]), metrics=metrics, opt_func=partial(RMSProp, wd=WD[0])).to_fp16()
learn.fine_tune(3) """

" CHOICE = 4\nmetrics=[FBetaMulti(BETA, THRESH[CHOICE], average='samples'), partial(accuracy_multi, thresh=THRESH[CHOICE])]\nlearn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=THRESH[CHOICE]), metrics=metrics, opt_func=partial(RMSProp, wd=WD[0])).to_fp16()\nlearn.fine_tune(3) "

In [28]:
""" CHOICE = 5
metrics=[FBetaMulti(BETA, THRESH[CHOICE], average='samples'), partial(accuracy_multi, thresh=THRESH[CHOICE])]
learn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=THRESH[CHOICE]), metrics=metrics, opt_func=partial(RMSProp, wd=WD[0])).to_fp16()
learn.fine_tune(3) """

" CHOICE = 5\nmetrics=[FBetaMulti(BETA, THRESH[CHOICE], average='samples'), partial(accuracy_multi, thresh=THRESH[CHOICE])]\nlearn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=THRESH[CHOICE]), metrics=metrics, opt_func=partial(RMSProp, wd=WD[0])).to_fp16()\nlearn.fine_tune(3) "

In [29]:
""" CHOICE = 6
metrics=[FBetaMulti(BETA, THRESH[CHOICE], average='samples'), partial(accuracy_multi, thresh=THRESH[CHOICE])]
learn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=THRESH[CHOICE]), metrics=metrics, opt_func=partial(RMSProp, wd=WD[0])).to_fp16()
learn.fine_tune(3) """

" CHOICE = 6\nmetrics=[FBetaMulti(BETA, THRESH[CHOICE], average='samples'), partial(accuracy_multi, thresh=THRESH[CHOICE])]\nlearn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=THRESH[CHOICE]), metrics=metrics, opt_func=partial(RMSProp, wd=WD[0])).to_fp16()\nlearn.fine_tune(3) "

Different Wd

In [30]:
""" CHOICE = 0
metrics=[FBetaMulti(BETA, THRESH[6], average='samples'), partial(accuracy_multi, thresh=THRESH[6])]
learn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=THRESH[6]), metrics=metrics, opt_func=partial(RMSProp, wd=WD[CHOICE])).to_fp16()
learn.fine_tune(3) """

" CHOICE = 0\nmetrics=[FBetaMulti(BETA, THRESH[6], average='samples'), partial(accuracy_multi, thresh=THRESH[6])]\nlearn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=THRESH[6]), metrics=metrics, opt_func=partial(RMSProp, wd=WD[CHOICE])).to_fp16()\nlearn.fine_tune(3) "

In [31]:
""" CHOICE = 1
metrics=[FBetaMulti(BETA, THRESH[6], average='samples'), partial(accuracy_multi, thresh=THRESH[6])]
learn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=THRESH[6]), metrics=metrics, opt_func=partial(RMSProp, wd=WD[CHOICE])).to_fp16()
learn.fine_tune(3) """

" CHOICE = 1\nmetrics=[FBetaMulti(BETA, THRESH[6], average='samples'), partial(accuracy_multi, thresh=THRESH[6])]\nlearn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=THRESH[6]), metrics=metrics, opt_func=partial(RMSProp, wd=WD[CHOICE])).to_fp16()\nlearn.fine_tune(3) "

In [32]:
""" CHOICE = 2
metrics=[FBetaMulti(BETA, THRESH[6], average='samples'), partial(accuracy_multi, thresh=THRESH[6])]
learn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=THRESH[6]), metrics=metrics, opt_func=partial(RMSProp, wd=WD[CHOICE])).to_fp16()
learn.fine_tune(3) """

" CHOICE = 2\nmetrics=[FBetaMulti(BETA, THRESH[6], average='samples'), partial(accuracy_multi, thresh=THRESH[6])]\nlearn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=THRESH[6]), metrics=metrics, opt_func=partial(RMSProp, wd=WD[CHOICE])).to_fp16()\nlearn.fine_tune(3) "

In [33]:
""" CHOICE = 3
metrics=[FBetaMulti(BETA, THRESH[6], average='samples'), partial(accuracy_multi, thresh=THRESH[6])]
learn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=THRESH[6]), metrics=metrics, opt_func=partial(RMSProp, wd=WD[CHOICE])).to_fp16()
learn.fine_tune(3) """

" CHOICE = 3\nmetrics=[FBetaMulti(BETA, THRESH[6], average='samples'), partial(accuracy_multi, thresh=THRESH[6])]\nlearn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=THRESH[6]), metrics=metrics, opt_func=partial(RMSProp, wd=WD[CHOICE])).to_fp16()\nlearn.fine_tune(3) "

In [34]:
""" CHOICE = 4
metrics=[FBetaMulti(BETA, THRESH[6], average='samples'), partial(accuracy_multi, thresh=THRESH[6])]
learn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=THRESH[6]), metrics=metrics, opt_func=partial(RMSProp, wd=5e-9)).to_fp16()
learn.fine_tune(3) """

" CHOICE = 4\nmetrics=[FBetaMulti(BETA, THRESH[6], average='samples'), partial(accuracy_multi, thresh=THRESH[6])]\nlearn = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=THRESH[6]), metrics=metrics, opt_func=partial(RMSProp, wd=5e-9)).to_fp16()\nlearn.fine_tune(3) "

Final

In [35]:
dblock = DataBlock(
    blocks=(ImageBlock,MultiCategoryBlock),
    splitter=RandomSplitter(valid_pct=0.3,seed=42),
    get_x=ColReader('path'),
    get_y=ColReader('label',label_delim=','),
    item_tfms = Resize(224),
    batch_tfms=aug_transforms(size=224,
                              do_flip=True,
                              flip_vert=False,
                              max_lighting=0.2,
                              max_warp=0.2,
                              p_lighting=0.75)
)
dls = dblock.dataloaders(df, bs=64,shuffle=True)

In [36]:
BETA = 2
THRESH = 0.125
WD = 5e-8
metrics=[FBetaMulti(BETA, THRESH, average='samples'),
          partial(accuracy_multi, thresh=THRESH)]

In [37]:
learn_final = vision_learner(dls, resnet50, loss_func=BCEWithLogitsLossFlat(thresh=THRESH), metrics=metrics, opt_func=partial(RMSProp, wd=WD)).to_fp16()
learn_final.fine_tune(3)

epoch,train_loss,valid_loss,fbeta_score,accuracy_multi,time
0,0.022126,0.020884,0.157350,0.994967,57:55


epoch,train_loss,valid_loss,fbeta_score,accuracy_multi,time
0,0.019654,0.018905,0.245897,0.994797,57:39
1,0.018782,0.018043,0.283665,0.994694,57:49
2,0.018403,0.017783,0.294880,0.994667,57:59


In [ ]:
learn_final.unfreeze()

In [ ]:
learn_final.lr_find()

In [ ]:
learn_final.fit_one_cycle(8, lr_max=slice(5e-7, 1e-4))

In [ ]:
learn_final.recorder.plot_loss()

In [ ]:
model = learn_final.model
model.cpu()
torch.save(model, 'attr_resnet50.pkl')

learn_final.export("attr_resnet50_export.pkl")

with open('attr_resnet50_open.pkl', 'wb') as file:
    pickle.dump(learn_final, file)

In [ ]:
learn_final.show_results()